In [1]:
import imaplib
import email
import pandas as pd
from bs4 import BeautifulSoup
# import re

from IPython.display import display, HTML
import time

# Get the current date as a string
DATE = time.strftime('%Y-%m-%d', time.localtime())
print(DATE)

2023-07-30


In [2]:

%%time
def fetch_email_messages(username, password, from_allow_list, to_allow_list):
    # Connect to the Gmail IMAP server
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(username, password)
    mail.select('inbox')
    _, data = mail.search(None, 'ALL')
    email_messages = []
    for num in data[0].split():
        _, msg_data = mail.fetch(num, '(RFC822)')
        email_message = email.message_from_bytes(msg_data[0][1])
        if email_message['From'] not in from_allow_list:
            print(f"Skipping email not in from_allow_list, From: {email_message['From']}")
            continue
        elif email_message['To'] not in to_allow_list:
            print(f"Skipping email not in to_allow_list, To: {email_message['To']}")
            continue
        email_messages.append(email_message)
    mail.logout()
    return email_messages

# Set your Gmail credentials and search query
# https://myaccount.google.com/u/3/apppasswords
password = "tfdxdxszsbmkuiiy"
username = "seattle.housing.feed@gmail.com"
from_list = [
    '"Zillow" <daily-updates@mail.zillow.com>',
    '"Zillow" <open-houses@mail.zillow.com>',
    'Redfin <listings@redfin.com>',
    '"Redfin" <redmail@redfin.com>'
]
to_list = ['seattle.housing.feed@gmail.com']
    
# Fetch emails and extract information
emails_messages = fetch_email_messages(username, password, from_list, to_list)
print(f'Fetched {len(emails_messages)} gmail messages to {username}')

Skipping email not in from_allow_list, From: Google Community Team <googlecommunityteam-noreply@google.com>
Skipping email not in from_allow_list, From: Jake G <omonoid@gmail.com>
Skipping email not in from_allow_list, From: "Redfin" <donotreply@redfin.com>
Skipping email not in from_allow_list, From: "Zillow" <no-reply@confirmation.zillow.com>
Skipping email not in from_allow_list, From: "Zillow" <reply@mail.zillow.com>
Skipping email not in from_allow_list, From: "Zillow" <reply@mail.zillow.com>
Skipping email not in from_allow_list, From: Google <no-reply@accounts.google.com>
Skipping email not in from_allow_list, From: The Google Account Team <google-noreply@google.com>
Skipping email not in from_allow_list, From: "Zillow" <no-reply@mail.zillow.com>
Skipping email not in from_allow_list, From: The Google Account Team <google-noreply@google.com>
Skipping email not in from_allow_list, From: Google <no-reply@accounts.google.com>
Skipping email not in from_allow_list, From: The Google 

In [4]:
%%time
def parse_email_message(email_message):
    email_res = {k:email_message[k] for k in ['Subject', 'To', 'From', 'Date']}
    from_name, from_email =  email_res['From'].split(' <')
    email_res['Sender'] = from_name.replace('"', '').strip()
    email_res['From'] = from_email.replace('>', '').strip()
    content = ''
    if email_message.is_multipart():
        for part in email_message.walk():
            content_type = part.get_content_type()
            if content_type == 'text/plain' or content_type == 'text/html':
                content += part.get_payload(decode=True).decode('UTF-8')
    else:
        content_type = email_message.get_content_type()
        if content_type == 'text/plain' or content_type == 'text/html':
            email_res['Content'] = email_message.get_payload(decode=True).decode('UTF-8')
    # email_res['Content'] = content
    bs4 = BeautifulSoup(content, 'html.parser')

    # Find all script tags and remove
    script_tags = bs4.find_all('script')
    for script_tag in script_tags:
        script_tag.decompose()
    # Find all the style tags and remove
    style_tags = bs4.find_all('style')
    for style_tag in style_tags:
        style_tag.extract()
    
    elements_to_remove = []
    if email_res['From'] == 'listings@redfin.com':
        elements_to_remove += bs4.find_all(class_='footer-layout-wrapper')
    elif email_res['From'] == 'redmail@redfin.com':
        elements_to_remove += bs4.find_all(class_='footer')
    elif (email_res['From'] == 'daily-updates@mail.zillow.com' or  
          email_res['From'] == 'open-houses@mail.zillow.com'):
        elements_to_remove = bs4.find_all('address')
        elements_to_remove += bs4.find_all(class_='dmTxtLinkSecondary')
    #     elements_to_remove = bs4.find_all(class_=lambda value: value and value.startswith('width100'))
    for element in elements_to_remove:
        element.extract()
        
    email_res['html'] = str(bs4.html)
    email_res['bs4'] = bs4    
    return email_res

emails = pd.DataFrame([parse_email_message(e) for e in emails_messages ])
emails['Date'] = pd.to_datetime(emails['Date'], format='%a, %d %b %Y %H:%M:%S %z')
emails['title'] = '<p style="text-align:center;">'+"<b>"+emails['Date'].astype(str)+"  :  "+emails['From']+"</b></p>"
emails['html'] = emails['title'] + emails['html']
emails = emails.sort_values('Date', ascending=False)

print(f'Parsed {len(emails)} gmail messages to {username}')
# display(pd.DataFrame(emails['From'].value_counts()))


Parsed 618 gmail messages to seattle.housing.feed@gmail.com
CPU times: user 16.4 s, sys: 708 ms, total: 17.1 s
Wall time: 17.2 s


In [5]:
%%time
header = """
<head>
  <title>Seattle Housing Inbox</title>
  <style>
      /* Styles for desktop */
      body {
          zoom: 150%;
          margin: 50px;
      }

      img {
          max-width: 100%;
          height: auto;
      }

      /* Styles for mobile */
      @media screen and (max-width: 600px) {
          body {
              zoom: 90%;
              margin: 5px;
          }
      }
  </style>
</head>
"""
html_file = 'index.html'
with open(html_file, 'w') as file:
  file.write(header + emails['html'].str.cat(sep='\n'))

CPU times: user 57.1 ms, sys: 79.9 ms, total: 137 ms
Wall time: 148 ms


In [ ]:
# import pdfkit
# pdf_file = html_file.replace('.html', '.pdf')
# pdfkit.from_file(html_file, pdf_file) #, options={'quiet': '', 'lowquality': ''}
# print("PDF file created:", pdf_file)